In [1]:
from NPSmethods import*
import numpy as np
from sklearn.preprocessing import StandardScaler
import glob
import pickle as pkl
import tensorflow as tf

In [2]:
def calcNoisePowSpecs(ODs, ODAvg, norm=False, imgSysData=None):

    noisePowSpecs = []
    for k in range(len(ODs)):
        noise = ODs[k]
        if norm:
            noise = OD2AtomNum(noise, imgSysData)
        noiseFFT = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(noise)))
        noisePowSpec = np.abs(noiseFFT)**2
        noisePowSpec = noisePowSpec / ODs[k].sum()
        noisePowSpec[noisePowSpec.shape[0]//2, noisePowSpec.shape[1]//2] = 0
        noisePowSpecs.append(noisePowSpec)

    return noisePowSpecs

In [3]:
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

choices = {
    "if_Save"                     : False,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : False,
    "plot_NoisePowSpec_LineCut"   : False,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

path = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image'
savepath = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\results'
numOfImgsInEachRun = 1
rampingParameter = 0
trapRegion = (slice(100, 200), slice(100, 200))
noiseRegion = (slice(0, 300), slice(0, 300))

In [4]:
parameters = []
images = []

for folder in glob.iglob(path + "\\*"):
    atomODs, atomODAvg, _, _, imgIndexMin, imgIndexMax = \
        readInImages(folder + "\\", numOfImgsInEachRun, rampingParameter, trapRegion, noiseRegion)
    noisePowSpecs = calcNoisePowSpecs(atomODs, atomODAvg, norm = False, imgSysData = imgSysData)
    _, _, _, _, popt, _ = doCalibration(folder, savepath, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
    for noisePowSpec in noisePowSpecs:
        images.append(noisePowSpec)
    for i in range(len(noisePowSpecs)):
        parameters.append(popt)

tau     =  0.9836 : describe the decaying of transmission efficiency with radius
S0      = -1.0744 : spherical aberration
alpha   =  1.2787 : astigmatism
beta    =  0.9343 : defocus
delta_s = -0.0909 : phase imprint by atom scattering

tau     =  1.0436 : describe the decaying of transmission efficiency with radius
S0      = -0.9551 : spherical aberration
alpha   =  0.9886 : astigmatism
beta    =  0.6105 : defocus
delta_s =  0.0956 : phase imprint by atom scattering

tau     =  0.6194 : describe the decaying of transmission efficiency with radius
S0      = -2.2355 : spherical aberration
alpha   =  0.0054 : astigmatism
beta    =  0.5476 : defocus
delta_s =  1.2896 : phase imprint by atom scattering

tau     =  0.6588 : describe the decaying of transmission efficiency with radius
S0      =  1.5097 : spherical aberration
alpha   =  0.3609 : astigmatism
beta    = -0.1034 : defocus
delta_s = -1.2108 : phase imprint by atom scattering

tau     =  0.7399 : describe the decaying of transmissio

In [5]:
images = np.array(images).reshape((len(images),-1))
parameters = np.array(parameters)[:, [3, 5, 6]]
yscaler = pkl.load(open(savepath + "\\yscaler", "rb"))
parameters = yscaler.transform(np.array(parameters))

In [6]:
images = tf.convert_to_tensor(images)
parameters = tf.convert_to_tensor(parameters)

In [7]:
pkl.dump(images, open(savepath + "\\realdatain", "wb"))
pkl.dump(parameters, open(savepath + "\\realdataout", "wb"))